In [ ]:
import pandas as pd
import scipy.stats as st
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df = pd.read_table ("resultados_diferentes.txt", sep='\s+', index_col = [1, 0])

In [ ]:
df["%dropS1"] = df.apply (lambda row: 100 * (float)(row["dropSlice1"]) / row["TxpacSlice1"], axis=1)
df["%dropS2"] = df.apply (lambda row: 100 * (float)(row["dropSlice2"]) / row["TxpacSlice2"], axis=1)
#df

In [ ]:
# Função para cálculo do intervalo de confiança de 95%
def confinterval (data):
    confidence = 0.95
    return st.sem (data) * st.t.ppf ((1 + confidence) / 2.0, len (data) - 1);

In [ ]:
df = df.query("numHosts != '67-33'")
gpb = df.groupby (["numHosts"])
media = gpb.mean()
err = gpb.aggregate(confinterval)
err = err.filter(items=['dropSlice1', 'dropSlice2'])
media = media.filter(items=['dropSlice1', 'dropSlice2'])
#display (media)
#display (err)

media.plot (kind='bar', yerr=err, title="Perda total de pacotes", rot=360, figsize=(10,5))
plt.xlabel('Numero de hosts em cada grupo de cada slice')
plt.ylabel('Quantidade de pacotes')
plt.gca().legend(('Pacotes perdidos no slice 1','Pacotes perdidos no slice 2'))
plt.savefig('grafico_hosts_diferentes.pdf')

In [ ]:
err = gpb.aggregate(confinterval)
media = gpb.mean()
err = err.filter(items=['%dropS1', '%dropS2'])
media = media.filter(items=['%dropS1', '%dropS2'])

media.plot (kind='bar', yerr=err, title="Perda total de pacotes (%)", rot=360, figsize=(10,5))
plt.xlabel('Numero de hosts em cada grupo de cada slice')
plt.ylabel('% de pacotes')
plt.gca().legend(('% Pacotes perdidos no slice 1','% Pacotes perdidos no slice 2'))
plt.savefig('grafico_hosts_diferentes_porcentagem.pdf')

In [ ]:
#Graficos para cada RNG

#lista_rng = ['1', '2','3','4']
#for i in lista_rng:
#    g=df[df.index.get_level_values('RNG').isin([i])].filter(items=['dropSlice1', 'dropSlice2'])
#    g.plot.bar(figsize=(20,10),title="Perda de pacotes, RNG = " + i, logy=True)

In [ ]:
#Graficos para cada numero de host

#lista_host = ['100-100','200-200','300-300','400-400','500-500','600-600']
#for i in lista_host:
#    g=df[df.index.get_level_values('numHosts').isin([i])].filter(items=['dropSlice1', 'dropSlice2'])
#    g.plot.bar(figsize=(20,10),title="Perda de pacotes, hosts = " + i)